# FuzzyWuzzy: Fuzzy String Matching/Comparison in Python, Deep Guide
## ... and a hands-on practice on a real-world dataset
<img src="images/repo.jpg"></img>

### Introduction

### Setup

In [1]:
# Load necessary libraries
import pandas as pd
# fuzzywuzzy to be imported later

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### How String Comparison Is Performed

### fuzzywuzzy: Installation

Even though the basic installation can be done easily with `pip`, there are some other options or caveats to `fuzzwuzzy`'s installation:

- Using PIP via PyPI (standard):

```pip install fuzzywuzzy```

The above method installs the default up-to-date version of the package. At first, I installed it using this method. But whenever I imported it, it started giving a warning saying that the package itself is very slow and I should install `python-Levenshtein` package for more speed. If you hate warnings in your Jupyter Notebook like me, here is how you can install extra dependencies:
- Directly install `python-Levenshtein`:

```pip install python-Levenshtein```

or

```pip install fuzzywuzzy[speedup]```
**Warning for Windows users**: if you don't have Microsoft Visual Studio build tools installed, installing `python-Levenshtein` fails. You can download it from [here](https://visualstudio.microsoft.com/downloads/).

### fuzzywuzzy: The Basics

### fuzzywuzzy: Comparison of Different Methods

### Text Cleaning With fuzzwuzzy on a real dataset